ML setup

- tracking server: yes
- backend store: sqlite database
- arrtifacts store: local filesystem

In [1]:
import mlflow

In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
print(f"Tracking uri: {mlflow.get_tracking_uri()}")

Tracking uri: http://127.0.0.1:5000


In [5]:
mlflow.list_experiments()

[<Experiment: artifact_location='./artifacts_local/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./artifacts_local/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2023/07/13 16:35:00 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: './artifacts_local/1/dd838d931f9a48ccbc3a28757799b140/artifacts'


### interact with model registry

In [8]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")

In [9]:
client.list_registered_models()

[]

In [14]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id

mlflow.register_model(model_uri=f"runs:/{run_id}/models", 
                     name="iris-classifier")

Successfully registered model 'iris-classifier'.
2023/07/13 16:40:00 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: creation_timestamp=1689262799993, current_stage='None', description='', last_updated_timestamp=1689262799993, name='iris-classifier', run_id='dd838d931f9a48ccbc3a28757799b140', run_link='', source='./artifacts_local/1/dd838d931f9a48ccbc3a28757799b140/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>